In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup
import time
import datetime

In [3]:
location = '대구광역시 중구 대봉동'
driver = webdriver.Chrome('./chromedriver.exe')
time.sleep(1)
driver.get('https://www.naver.com/') # 네이버로 접속
time.sleep(0.5)
input_location = driver.find_element(By.XPATH, '//*[@id="query"]') # 검색창 위치 찾기
input_location.send_keys(f'{location} 날씨') # 날씨값 입력
time.sleep(0.1)
input_location.submit() # 검색 전송
time.sleep(0.1)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

C:\Users\User\AppData\Local\Temp\ipykernel_24560\2616694854.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


# 온도

In [3]:
temperature = soup.select('div.point_box>span.num')[0:8]
temperature

[<span class="num">34<span class="celsius">°</span></span>,
 <span class="num">33<span class="celsius">°</span></span>,
 <span class="num">32<span class="celsius">°</span></span>,
 <span class="num">30<span class="celsius">°</span></span>,
 <span class="num">29<span class="celsius">°</span></span>,
 <span class="num">29<span class="celsius">°</span></span>,
 <span class="num">28<span class="celsius">°</span></span>,
 <span class="num">27<span class="celsius">°</span></span>]

In [73]:
for i in temperature:
    print(i.text[-3:-1])

30
31
32
32
32
33
32
31


# 강수확률

In [18]:
soup.select('div.icon_wrap>ul>li>em.value')[0:7][0].text

'-'

# 습도

In [89]:
soup.select('div.humidity_graph_box>div>div>div>div>ul>li>div>span>span')[0:7]

[<span class="num">55</span>,
 <span class="num">60</span>,
 <span class="num">75</span>,
 <span class="num">80</span>,
 <span class="num">80</span>,
 <span class="num">80</span>,
 <span class="num">80</span>]

In [34]:
len(soups[0].select('span.num'))/4

126.0

In [49]:
soups[0].select('span.num')[120]

<span class="num">85</span>

In [31]:
soups[0].select('span.num')[-58:-50]
# 기상청 데이터

[<span class="num">2</span>,
 <span class="num">1</span>,
 <span class="num">50</span>,
 <span class="num">60</span>,
 <span class="num">70</span>,
 <span class="num">80</span>,
 <span class="num">80</span>,
 <span class="num">80</span>]

In [44]:
soups[1].select('span.num')[-58:-50]
# 아큐웨더 데이터

[<span class="num">1</span>,
 <span class="num">35</span>,
 <span class="num">40</span>,
 <span class="num">40</span>,
 <span class="num">55</span>,
 <span class="num">60</span>,
 <span class="num">65</span>,
 <span class="num">65</span>]

In [56]:
soups[2].select('span.num')[-73:-65]
# 웨더채널 데이터

[<span class="num">1</span>,
 <span class="num">47</span>,
 <span class="num">48</span>,
 <span class="num">50</span>,
 <span class="num">54</span>,
 <span class="num">59</span>,
 <span class="num">64</span>,
 <span class="num">69</span>]

In [61]:
soups[3].select('span.num')[-49:-41]

[<span class="num">3</span>,
 <span class="num">43</span>,
 <span class="num">43</span>,
 <span class="num">47</span>,
 <span class="num">52</span>,
 <span class="num">57</span>,
 <span class="num">70</span>,
 <span class="num">71</span>]

span에서 class가 num인 것만 추출하면 날씨, 강수, 바람, 습도의 데이터가 나오는데, 습도는 제일 뒷 부분 인것을 활용  
weather, aqweather는 [-57:-50] 슬라이싱을 활용

In [4]:
def Crawler(location):
    soups = []
    driver = webdriver.Chrome('./chromedriver.exe')
    time.sleep(1)
    driver.get('https://www.naver.com/') # 네이버로 접속
    time.sleep(0.5)
    input_location = driver.find_element(By.XPATH, '//*[@id="query"]') # 검색창 위치 찾기
    input_location.send_keys(f'{location} 날씨') # 날씨값 입력
    time.sleep(0.1)
    input_location.submit() # 검색 전송
    time.sleep(0.1)
    html = driver.page_source
    soup_wether = BeautifulSoup(html, 'html.parser')
    soups.append(soup_wether)
    time.sleep(3) # 페이지가 늦게 렌더링 되었을때 오류가 나오는데, 이를 방지
    driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/a/strong').click() # 화살표 아래로
    time.sleep(0.1)
    driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/div/div/div/div/ul/li[2]/a').click() # 아큐웨더
    html = driver.page_source
    soup_aqweather = BeautifulSoup(html, 'html.parser')
    time.sleep(0.1)
    soups.append(soup_aqweather)
    driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/a/strong').click() # 화살표 아래로
    time.sleep(0.1)
    driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/div/div/div/div/ul/li[3]/a').click() # 웨더채널
    html = driver.page_source
    soup_weatherchannel = BeautifulSoup(html, 'html.parser')
    soups.append(soup_weatherchannel)
    time.sleep(0.1)
    driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/a/strong').click() # 화살표 아래로
    time.sleep(0.1)
    driver.find_element(By.XPATH, '//*[@id="main_pack"]/section[1]/div[1]/div[2]/div[1]/div[1]/div/div[1]/div[2]/div/div/div/div/div/ul/li[4]/a').click() # 웨더뉴스
    html = driver.page_source
    soup_weathernews = BeautifulSoup(html, 'html.parser')
    soups.append(soup_weathernews)
    
    return soups

In [5]:
soups = Crawler('대구광역시 중구 대봉동')

C:\Users\User\AppData\Local\Temp\ipykernel_24560\2256287994.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [12]:
def Temperature(soups):
    temperatures = {}
    for i in range(4):
        temp = []
        for j in soups[i].select('div.point_box>span.num')[0:7]:# 7시간 동안의 기온 추출
            temp.append(int(j.text[-3:-1]))
        if i == 0:
            temperatures['weather'] = temp
        elif i == 1:
            temperatures['aqweather'] = temp
        elif i == 2:
            temperatures['weatherchannel'] = temp
        elif i == 3:
            temperatures['weathernews'] = temp
    return temperatures

In [13]:
t = Temperature(soups)
t

{'weather': [29, 28, 27, 25, 24, 24, 24],
 'aqweather': [29, 28, 27, 25, 24, 24, 24],
 'weatherchannel': [29, 29, 28, 27, 26, 25, 24],
 'weathernews': [23, 25, 27, 26, 25, 25, 24]}

In [14]:
def Rainprobability(soups):
    rainprobabilities = {}
    for i in range(4):
        temp = []
        for j in soups[i].select('div.icon_wrap>ul>li>em.value')[0:7]:
            if j.text == '-':
                temp.append(0)
            else:
                temp.append(int(j.text[:-1]))
        if i == 0:
            rainprobabilities['weather'] = temp
        elif i == 1:
            rainprobabilities['aqweather'] = temp
        elif i == 2:
            rainprobabilities['weatherchannel'] = temp
        elif i == 3:
            rainprobabilities['weathernews'] = temp
            
    return rainprobabilities

In [15]:
r = Rainprobability(soups)
r

{'weather': [0, 0, 0, 0, 30, 30, 30],
 'aqweather': [0, 0, 0, 0, 30, 30, 30],
 'weatherchannel': [61, 49, 20, 20, 20, 20, 20],
 'weathernews': [71, 71, 63, 66, 66, 59, 37]}

In [92]:
def Humidity(soups):
    humidities = {}
    for i in range(4):
        temp = []
        for j in soups[i].select('div.humidity_graph_box>div>div>div>div>ul>li>div>span>span')[0:7]:
            temp.append(int(j.text))
        if i == 0:
            humidities['weather'] = temp
        elif i == 1:
            humidities['aqweather'] = temp
        elif i == 2:
            humidities['weatherchannel'] = temp
        elif i == 3:
            humidities['weathernews'] = temp
    return humidities

In [93]:
h = Humidity(soups)
h

{'weather': [50, 60, 70, 80, 80, 80, 80],
 'aqweather': [50, 60, 70, 80, 80, 80, 80],
 'weatherchannel': [64, 68, 73, 74, 76, 79, 81],
 'weathernews': [81, 74, 65, 71, 75, 79, 81]}

## 기상청, aqweather 현재 날씨

In [42]:
soups[0].select('dd.desc')[2].text
# 풍속

'2.4m/s'

In [43]:
soups[0].select('dt.term')[2].text
#풍향

'바람(북풍)'

In [31]:
soups[1].select('dd.desc')[2].text
# 풍속

'2.4m/s'

In [32]:
soups[1].select('dt.term')[2].text
#풍향

'바람(북풍)'

In [56]:
soups[1].select('dt.term')[2].text + ':' + soups[1].select('dd.desc')[2].text

'바람(북풍):2.4m/s'

## weatherchannel 현재날씨

In [44]:
soups[2].select('dd')[2].text

'2.8m/s'

In [45]:
soups[2].select('dt')[2].text

'바람(북서풍)'

In [46]:
soups[3].select('dd')[2].text

'2m/s'

In [47]:
soups[3].select('dt')[2].text

'바람(북북서풍)'

In [51]:
import datetime

In [66]:
now = datetime.datetime.now()

'2022-720'

In [29]:
def now(location, soups):
    state = {}
    now = datetime.datetime.now()
    weekdays = ['월', '화', '수', '목', '금', '토', '일']
    state['year'] = now.year
    state['month'] = now.month
    state['day'] = now.day
    state['location'] = location
    state['hour'] = now.hour
    state['minute'] = now.minute
    state['weekday'] = weekdays[now.weekday()]
    t = Temperature(soups)
    r = Rainprobability(soups)
    h = Humidity(soups)
    state['temperature'] = round((int(t['weather'][0]) + t['aqweather'][0] + t['weatherchannel'][0] + t['weathernews'][0])/4, 1)
    state['rain_prob'] = round((r['weather'][0] + r['aqweather'][0] + r['weatherchannel'][0] + r['weathernews'][0])/4, 1)
    state['humidity'] = round((h['weather'][0] + h['aqweather'][0] + h['weatherchannel'][0] + h['weathernews'][0])/4, 1)
    state['wind'] = round((float(soups[0].select('dd.desc')[2].text[:-3]) + float(soups[1].select('dd.desc')[2].text[:-3]) + float(soups[2].select('dd')[2].text[:-3]) + float(soups[3].select('dd')[2].text[:-3]))/4,1)
    return state

In [21]:
location

'대구광역시 중구 대봉동'

In [94]:
now(location, soups)

{'year': 2022,
 'month': 7,
 'day': 22,
 'location': '대구광역시 중구 대봉동',
 'hour': 17,
 'minute': 0,
 'weekday': '금',
 'temperature': 27.5,
 'rain_prob': 33.0,
 'humidity': 61.2,
 'wind': 1.8}